In [ ]:
### import json
import pandas as pd
import MeCab
import matplotlib.pyplot as plt
import csv
import numpy as np
from wordcloud import WordCloud
from janome.analyzer import Analyzer
from janome.charfilter import UnicodeNormalizeCharFilter, RegexReplaceCharFilter
from janome.tokenizer import Tokenizer as JanomeTokenizer  # sumyのTokenizerと名前が被るため
from janome.tokenfilter import POSKeepFilter, ExtractAttributeFilter
import re
import ipadic

import unicodedata
#Algos
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.reduction import ReductionSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from janome.tokenizer import Tokenizer as JanomeTokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
from sumy.utils import get_stop_words

import pkg_resources, imp
imp.reload(pkg_resources)

import spacy
nlp = spacy.load('ja_ginza')#spacyの日本語モデル(Ginza)をロード

from ginza import *
import neologdn
import re
import emoji
import mojimoji
from collections import Counter

In [6]:
import sys
import pandas as pd
#
import sqlite3
# ------------------------------------------------------------------
sys.stderr.write("*** 開始 ***\n")
#読み込み先
file_company = "./datas/OpenWork_company.db"
file_company_info = './datas/OpenWork_company_info.db'

conn = sqlite3.connect(file_company)
conn2 = sqlite3.connect(file_company_info)
#
company_df=pd.read_sql_query('SELECT * FROM df_company', conn)
company_info_df=pd.read_sql_query('SELECT * FROM df_company_info', conn2)
display(company_df.head(15))
display(company_info_df.head(15))
#
conn.close()
conn2.close()
sys.stderr.write("*** 終了 ***\n")

#複数行に値が重複している可能性があるためdrop_duplicatesで、1行1ユニーク値にする
company_df = company_df.drop_duplicates()
company_info_df = company_info_df.drop_duplicates()

*** 開始 ***


,企業名,企業ID,口コミ種別ID,回答日時,評価,本文,業種,口コミ種別,職種,経験年数,現職/退職,新卒/中途,性別
0,株式会社極洋,a0910000000Fqdi,1,2021年09月16日,2.9,典型的な昔の会社。上の人間になればなるほど安定を好み、チャレンジ精神や新しい文化を取り入れる...,食品、飲料,組織体制・企業文化,水産,営業,在籍3年未満,現職（回答時）,新卒入社
1,株式会社極洋,a0910000000Fqdi,1,2018年04月12日,3.8,一部上場企業なので土台はしっかりしている。歴史も80年以上あるので、それだけ続いてきた会社な...,食品、飲料,組織体制・企業文化,管理部門,在籍3～5年,退社済み(2015年より前),新卒入社,女性
2,株式会社極洋,a0910000000Fqdi,1,2017年05月17日,2.8,水産、加工食品、常温、鰹鮪の４事業が主力。本社、支社に分けられる。支社が販社、独立採算制。人...,食品、飲料,組織体制・企業文化,営業,在籍10～15年,退社済み(2020年より前),新卒入社,男性
3,株式会社極洋,a0910000000Fqdi,1,2018年03月05日,3.3,良い意味では会社が古く、福利厚生・給与等安定している。また、基本的にクビにもならない。悪い意...,食品、飲料,組織体制・企業文化,社員,在籍3～5年,現職（回答時）,新卒入社,男性
4,株式会社極洋,a0910000000Fqdi,1,2018年01月04日,2.6,古い会社なので年功序列、上の言うことは絶対の風潮があり、風通しは悪い。また年齢構成がいびつに...,食品、飲料,組織体制・企業文化,営業,在籍5～10年,現職（回答時）,新卒入社,男性
5,株式会社極洋,a0910000000Fqdi,1,2015年07月17日,3.3,水産気質というか、たくましい人が多い。上下関係は厳しかったが、根が優しい人が多かった印象。実...,食品、飲料,組織体制・企業文化,営業,在籍3～5年,退社済み(2015年より前),新卒入社,男性
6,日本水産株式会社,a0910000000Fqdk,1,2016年03月25日,2.0,終身雇用・年功序列・新卒偏重・男性社会などの昭和の遺産から脱却できていない。それらに当てはま...,食品、飲料,組織体制・企業文化,事務,在籍5～10年,現職（回答時）,中途入社,女性
7,日本水産株式会社,a0910000000Fqdk,1,2017年04月12日,2.9,体育会系かと思います。お客様が第一、上司が第一。食品と水産とファイン事業があるので部署により...,食品、飲料,組織体制・企業文化,マーケティング,在籍5～10年,現職（回答時）,新卒入社,女性
8,日本水産株式会社,a0910000000Fqdk,1,2016年09月19日,2.4,社長が交代して変わった可能性があるが、「臭いものには蓋をする」という風土があったと思う。何か...,食品、飲料,組織体制・企業文化,管理部門,在籍3～5年,退社済み(2015年より前),中途入社,女性
9,日本水産株式会社,a0910000000Fqdk,1,2016年03月03日,2.8,創業100年を超えている老舗企業であるので、組織体制及び企業文化については良くも悪くも老舗の...,食品、飲料,組織体制・企業文化,営業,在籍10～15年,退社済み(2015年より前),新卒入社,男性


,企業名,月残業時間,有給消化率,平均年収,設立年,総合スコア,社員の士気,風通し,社員相互尊重,20代成長性,長期育成,コンプラ,評価納得感,待遇満足度,業種
0,シークス,16.9,36.9,None,1937.0,3.00,3.3,2.9,2.4,3.1,2.5,3.2,3.2,3.5,食品、飲料
1,日本水産株式会社,31.1,59.0,None,1943.0,2.98,2.9,2.9,2.5,2.6,2.3,4.1,4.1,3.5,食品、飲料
2,マルハニチロ株式会社,16.2,49.3,530,1943.0,3.15,2.9,3.1,2.9,3.2,2.9,4.4,4.4,2.9,食品、飲料
3,株式会社ホウスイ,None,None,None,1945.0,2.93,2.9,2.9,2.9,3.0,2.8,2.9,2.9,3.2,食品、飲料
4,株式会社雪国まいたけ,14.3,46.6,None,2015.0,2.48,2.4,2.4,2.5,2.2,2.3,2.7,2.7,2.7,食品、飲料
5,カネコ種苗株式会社,20.2,20.0,None,1947.0,2.76,2.8,2.4,2.8,2.5,2.2,3.6,3.6,2.8,農業、林業、水産、畜産
6,株式会社サカタのタネ,10.7,54.8,None,1942.0,3.20,2.8,3.6,3.1,2.6,2.9,4.0,4.0,3.6,農業、林業、水産、畜産
7,ホクト株式会社（食品）,21.4,58.2,423,1964.0,2.85,3.0,2.7,3.4,2.6,2.6,3.2,3.2,2.7,食品、飲料
8,株式会社ホクリヨウ,29.0,37.0,None,1949.0,2.89,2.9,2.9,2.8,2.7,2.9,3.0,3.0,3.0,食品、飲料
9,株式会社ヒノキヤグループ,32.2,39.6,None,1988.0,2.89,2.9,3.0,2.9,2.9,2.9,3.0,3.0,2.7,不動産関連、住宅


*** 終了 ***


In [7]:
import numpy as np
import pandas as pd

#前処理
company_df.本文 = company_df.本文.str.replace('\u3000', ' ')
company_df.本文 = company_df.本文.str.replace('■', ' ')
company_df.本文 = company_df.本文.str.replace('⇒', ' ')
company_df.本文 = company_df.本文.str.replace('→', ' ')
company_df.本文 = company_df.本文.str.replace('□', ' ')
company_df.本文 = company_df.本文.str.replace('：', ' ')



def preprocessing(company_info_df):
    
    #まず、'--'が含まれている場合はNaNに置き換える
    replace_line = company_info_df.replace('--',None)

    #欠損していない値が3未満の企業はdrop
    processed_company_info_df = replace_line.dropna(axis=1, thresh=3,  inplace=False)

    #obj→floatに変換
    processed_company_info_df.月残業時間 = processed_company_info_df.月残業時間.astype('float')
    processed_company_info_df.有給消化率 = processed_company_info_df.有給消化率.astype('float')
    processed_company_info_df.平均年収  = processed_company_info_df.平均年収.astype('float')
    processed_company_info_df.総合スコア = processed_company_info_df.総合スコア.astype('float')
    processed_company_info_df.社員の士気 = processed_company_info_df.社員の士気.astype('float')
    processed_company_info_df.風通し = processed_company_info_df.風通し.astype('float')
    processed_company_info_df.社員相互尊重 = processed_company_info_df.社員相互尊重.astype('float')
    processed_company_info_df['20代成長性'] = processed_company_info_df['20代成長性'].astype('float')
    processed_company_info_df.長期育成 = processed_company_info_df.長期育成.astype('float')
    processed_company_info_df.コンプラ = processed_company_info_df.コンプラ.astype('float')
    processed_company_info_df.評価納得感 = processed_company_info_df.評価納得感.astype('float')
    processed_company_info_df.待遇満足度 = processed_company_info_df.待遇満足度.astype('float')
    
    #Nullは中央値で埋める
    processed_company_info_df = processed_company_info_df.fillna(value=processed_company_info_df.median())
    
    #その他、分析に必要なdfの必要な前処理
    processed_company_info_df['会社年齢'] = 2022 - processed_company_info_df['設立年']
    processed_company_info_df.drop('設立年',axis=1,inplace=True)
    
    return processed_company_info_df


company_indo_df = preprocessing(company_info_df)

In [4]:
#stopword list 
stopword_list = []
with open('C:/Users/Administrator/Desktop/Python 3/Graduation Task/stopwords/stopwords.csv','r',encoding='utf-8') as f:
    text = f.readlines()
    for tk in text:
        tk = tk.strip('\n')
        tk = tk.replace('\ufeff','')
        stopword_list.append(tk)


In [8]:
#Mecabアナライザ
def mecab_analyzer(text):
    #正規化
    text = unicodedata.normalize("NFKC", text)
    #text = re.sub(r'\u3000','',text)
    text.replace('\u3000', '')
    tagger = MeCab.Tagger("-Owakati")
    node = tagger.parseToNode(text)

    # 指定した品詞を抽出しリストに
    word_list = []
    
    while node:
        word_type = node.feature.split(',')[0]
        if word_type in ['名詞', '形容詞', '動詞','副詞','接尾辞']:
            word_list.append(node.surface)
        #ここのインデントミスると無限ループ
        node = node.next
    # リストを文字列に変換
    word_chain = ' '.join(word_list)
    return word_list


In [13]:
#spacyアナライザ
import spacy
import ginza
nlp = spacy.load('ja_ginza')
ginza.set_split_mode(nlp, "C") # 形態素の分割モード指定

def spacy_analyzer(text):
    #正規化
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r'\u3000','',text)
    text.replace('\u3000', '')
    doc = nlp(text)

    #結果を格納するリスト
    result_list = []
    for sent in doc.sents:
        for token in sent:
            if token.pos_ in ['NOUN', 'ADJ', 'VERB','ADV']:#名詞、形容詞、動詞、副詞のみ
                result_list.append(token.text)
            
        #https://yu-nix.com/blog/2021/3/3/spacy-pos-list/#spaCy%E3%81%AE%E5%93%81%E8%A9%9E%E4%B8%80%E8%A6%A7%EF%BC%88%E6%97%A5%E6%9C%AC%E8%AA%9E%E8%A8%B3%EF%BC%89
        #result_list = result_list + [token.text for token in sent if token.pos_ in ['NOUN', 'ADJ', 'VERB','ADV','SCONJ','PART','DET','AUX']]

    return result_list

In [14]:
import MeCab
from gensim.models import KeyedVectors
import numpy as np
from gensim.models import word2vec
import logging
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [ ]:
#word2vecを使った類似性判定
#まずは入手したテキストデータから、モデルを作成する(訓練済モデルは使わない)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#モデル作成
# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。
model = word2vec.Word2Vec(company_df["本文"].apply(mecab_analyzer), window=5,min_count=1,size=200,iter=100,sg=1)#sg=1:skipgram sg=0:CBOW
model.save("./model/w2v_nottrained_skip.model")

In [ ]:
#model load
import gensim
model = gensim.models.KeyedVectors.load('./model/w2v_nottrained_skip.model')

In [147]:
# テキストのベクトルを計算
def get_vector(text):
    sum_vec = np.zeros(200)
    word_count = 0
    mecab = MeCab.Tagger('-Owakati')
    node = mecab.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        # 名詞、動詞、形容詞に限定
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞' or fields[0] == '副詞' or fields[0] == '接尾辞':
            sum_vec += model[node.surface]
            word_count += 1
        node = node.next
    return sum_vec / word_count#全単語ベクトルの平均をとっているだけ

In [148]:
allsentence_processed = company_df["本文"].apply(mecab_analyzer).apply(' '.join)

In [149]:
all_sp = pd.DataFrame(allsentence_processed)

In [150]:
array_list = []
for i in range(all_sp.shape[0]):
    try:
        array = get_vector(all_sp.本文.iloc[i])
    except KeyError:
        array = np.zeros(200)
    finally:
        array_list.append(array)
allsp_array = np.array(array_list)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [158]:
allsp_array.shape
np.save('./array/w2v.pkl', allsp_array, allow_pickle=True)

In [159]:
#load tfidf matrix
tfidf_matrix = np.load('./array/tfidf_matrix.pkl.npy', allow_pickle=True, encoding='ASCII')
tfidf_matrix

array(<39285x7446 sparse matrix of type '<class 'numpy.float64'>'
	with 1443337 stored elements in Compressed Sparse Row format>,
      dtype=object)

In [160]:
#load
allsp_array = np.load('./array/w2v.pkl.npy',allow_pickle=True)

In [161]:
#dropna
#df化
allsp_df = pd.DataFrame(allsp_array)

#dropna(全カラムNAならば(KeyErrorになっていれば)、その行を削除)
#allsp_df.dropna(how='all',axis=0)

#ndarrayに戻す
allsp_array_fixed = np.array(allsp_df.fillna(0,axis=1))

In [162]:
#test run
results = model.most_similar(positive="若手", topn=10)
for result in results:
    print(result)

('若い', 0.6538199186325073)
('プロジェク', 0.483980655670166)
('若年', 0.47975876927375793)
('新人', 0.47425803542137146)
('アラフォー', 0.4680370092391968)
('中堅', 0.4588540196418762)
('居眠り', 0.4559928774833679)
('カルチァー', 0.449979305267334)
('若く', 0.4451788365840912)
('引き上げよう', 0.4433352053165436)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [163]:
from sklearn.metrics.pairwise import cosine_similarity
text = '若手が活躍している'
text_vector = get_vector(text)

#文書間のcos類似度を計算 
text_similarity = cosine_similarity(text_vector.reshape(-1,200), allsp_array_fixed)

#しきい値0.3以上を類似ベクトルとして抽出
num_of_similarities = (np.sum([x > 0.3 for x in text_similarity]))

#類似度でsortとargsortして、類似度とインデックス双方を取り出す topn件
top=20
top_indices = np.argsort(-text_similarity)[0][:num_of_similarities].tolist()[:top+1]
top_similarity = np.sort(text_similarity.ravel())[::-1][:num_of_similarities].tolist()[:top+1]
    
#dfのハコを作り
answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    
#カラム内の文字数。デフォルトは50なので変更
pd.set_option("display.max_colwidth", 1500)
    
#行数上限も変更し
pd.set_option("display.max_rows", 101)

#forループで連結する
for a,b in zip(top_indices,top_similarity):
    index_data = company_df.iloc[a]
    answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)
        
    
#出来上がったdfを、company_info_dfとマージ
answer_df_marged = pd.merge(answer_df,pd.DataFrame(company_info_df[['企業名','総合スコア']]),on='企業名',how='inner')
    
#企業風土の評価スコアはdrop
#answer_df_marged = answer_df_marged.drop('評価',axis=1)
#出来上がったdfを、類以度でsort　topは表示件数

answer_df_marged_g = answer_df_marged.set_index(['総合スコア','企業名'])

answer_df_marged[:top]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


,類似度,企業名,評価,本文,職種,経験年数,現職/退職,新卒/中途,性別,総合スコア
0,0.777144,株式会社INPEX（旧：国際石油開発帝石株式会社）,3.4,非常に働きやすい環境である。海外駐在になるチャンスが多い。優秀な若手社員が活躍している一方で、年功序列がまだまだ残っている。,技術系,在籍5～10年,現職（回答時）,中途入社,男性,3.04
1,0.770468,株式会社トリドールホールディングス,3.4,やりたいことが明確にある人は挑戦するチャンスがある。若手が活躍できる。20～40代の社員が多い。,教育,在籍3～5年,現職（回答時）,新卒入社,女性,2.94
2,0.766813,エイベックス株式会社（音楽）,3.4,若い人が活躍しやすい空気はあった,制作宣伝,宣伝,一般社員,在籍3～5年,退社済み(2015年より前),2.70
3,0.754010,損害保険ジャパン株式会社,3.9,基本的に年功序列ではあるが、会社はそういった文化を変えようとしている。あまり能力があるとは言えなそうな高齢社員を出向させたり、若手の育成に力を入れている。また、女性活躍を推進している。個人的には女性活躍＝女性のコスパが良いから と思っているが、今後は給与体系も男女平等になるよう。,ディーラー営業,営業,主任,在籍5～10年,現職（回答時）,3.41
4,0.753408,トルク株式会社,2.3,何年か前から新卒採用に力を入れて来たことで若手社員の比率、女性社員の比率が高い。20代後半、30代前半の中堅社員が抜けた穴を若手で埋めている。そのため、若手にもある程度の裁量を与えており活躍の場はあると思います。,営業,在籍3年未満,現職（回答時）,新卒入社,男性,2.83
5,0.749386,株式会社サイバーエージェント,3.4,社内で掲げ、また対外的にアピールしている通り、若手の台頭を喜ぶ社風があります。よくも悪くも若手に大きな仕事が任されるところがあります。その一方で中途入社や若手と呼ばれる年齢からはずれた社員は相対的に活躍しづらい環境があるように思います。,AI事業部,エンジニア,在籍5～10年,現職（回答時）,中途入社,4.00
6,0.741069,スズデン株式会社（電子）,2.9,全体的に人を大切にしているとは思います。社員の高齢化が進んでおり、若手が辞めないような企業努力は感じられました。業界としても若手のいくせいは課題になると思います。,法人,営業,在籍5～10年,退社済み(2020年より前),新卒入社,2.84
7,0.740132,ユニ・チャーム株式会社,3.1,上下関係がはっきりしており、体育会のノリが存在している。出る杭は活かすという方針はあるものの実際に活用されているケースは稀で若手社員が伸び伸びと仕事したり、自由闊達に活躍している印象は薄い。,国内,在籍5～10年,退社済み(2020年より前),新卒入社,男性,3.51
8,0.737426,三井住友海上火災保険株式会社,3.5,役割イノベーションとして地域職の業務領域の拡大を図り、総合職の役割の変革を押し進めているが、体制作りは各支店に委ねられており、支店によっては別会社のよう。縦割り組織であり、横の連携が薄い。但し、人材育成の各種研修が充実しており、また住宅手当をはじめ福利厚生は非常に充実している。事業としては、海外、企業分野に資源を積極的に投資しており、海外勤務をするチャンスは今後更に増えていくことが予想される。20代で海外勤務している者もいるが、まだ少ないのが現状。また女性の活躍推進にも力を入れている。やる気のある女性にとっては良い環境。しかし実際には昔に事務職で入社した女性が多く、営業をしていくという会社の方針に不満を抱き、できるだけ楽をしたいという人も多いのが現状。,営業,在籍5～10年,退社済み(2020年より前),新卒入社,男性,3.32
9,0.728865,三井住友海上火災保険株式会社,3.1,年功序列、男尊女卑ながらも女性活躍、若手活躍を掲げようとしてチグハグになっている印象。デジタル化の波に乗ろうと、新しいことをしようとする文化はありますが、取り入れる側が古い。ただ、産休、育休、時短等を活用する女性にとっては働きやすい環境であることは間違いないと思う。若手男性は朝から晩まで課長職の手となり足となりつかわれている上下関係がしっかりしている。,本社部門,事務,在籍5～10年,現職（回答時）,新卒入社,3.32


In [93]:
#訓練済モデル使ってみる
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./model/entity_vector.model.bin', binary=True)

2022-04-04 21:23:53,585 : INFO : loading projection weights from ./model/entity_vector.model.bin
2022-04-04 21:24:05,036 : INFO : loaded (1015474, 200) matrix from ./model/entity_vector.model.bin


In [94]:
results = model.most_similar(u'[ホワイト]')
for result in results:
    print(result)

2022-04-04 21:24:05,066 : INFO : precomputing L2-norms of word weight vectors


('[グレー]', 0.7590053081512451)
('ピュアホワイト', 0.7589070796966553)
('パールホワイト', 0.7467074394226074)
('サーモンピンク', 0.7321293950080872)
('ブリンドル', 0.7203001976013184)
('[桜色]', 0.7165554761886597)
('オリーブドラブ', 0.7147687673568726)
('オフホワイト', 0.7088093757629395)
('アイシャドウ', 0.7055991291999817)
('白緑', 0.7036178112030029)


In [116]:
from tqdm import tqdm
array_list_trained = []
for i in tqdm(range(all_sp.shape[0])):
    try:
        array = get_vector(all_sp.本文.iloc[i])
    except KeyError:
        array = np.zeros(200)
    finally:
        array_list_trained.append(array)

 30%|██████████████████████▊                                                    | 11944/39285 [01:29<03:54, 116.70it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
100%|███████████████████████████████████████████████████████████████████████████| 39285/39285 [05:03<00:00, 129.45it/s]


In [117]:
len(array_list_trained)
np.array(array_list_trained).shape

(39285, 200)

In [118]:
#リストからarrayに
allsp_array_trained = np.array(array_list_trained)

#df化
allsp_df_trained = pd.DataFrame(allsp_array_trained)

#dropna(いずれかの列に一個でもNAがあれば、その行を削除)
#allsp_df_trained.dropna(how='any',axis=0)

#ndarrayに戻す
allsp_array_trained = np.array(allsp_df_trained.dropna(how='any',axis=0))

allsp_array_trained.shape

(39281, 200)

In [119]:
text = '残業が少なく、働きやすい'
text_vector_trained = get_vector(text)

#文書間のcos類似度を計算
text_similarity = cosine_similarity(text_vector_trained.reshape(-1,200), allsp_array_trained)

#しきい値0.3以上を類似ベクトルとして抽出
num_of_similarities = (np.sum([x > 0.3 for x in text_similarity]))

#類似度でsortとargsortして、類似度とインデックス双方を取り出す top n件
top = 20
top_indices = np.argsort(-text_similarity)[0][:num_of_similarities].tolist()[:top+1]
top_similarity = np.sort(text_similarity.ravel())[::-1][:num_of_similarities].tolist()[:top+1]
    
#dfのハコを作り
answer_df = pd.DataFrame(columns=['類似度','企業名', '評価', '本文', '職種', '経験年数','現職/退職','新卒/中途','性別'])
    
#カラム内の文字数。デフォルトは50なので変更
pd.set_option("display.max_colwidth", 1500)
    
#行数上限も変更し
pd.set_option("display.max_rows", 101)

#forループで連結する
for a,b in zip(top_indices,top_similarity):
    index_data = company_df.iloc[a]
    answer_df =answer_df.append({'類似度': b,'企業名': index_data[0], 
                                     '評価': index_data[4],'本文':index_data[5],
                                     '職種':index_data[8],'経験年数':index_data[9],
                                     '現職/退職':index_data[10],'新卒/中途':index_data[11],
                                     '性別':index_data[12]},ignore_index=True)
        
    
#出来上がったdfを、company_info_dfとマージ
answer_df_marged_g = pd.merge(answer_df,pd.DataFrame(company_info_df[['企業名','総合スコア']]),on='企業名',how='inner')
    
#企業風土の評価スコアはdrop
#answer_df_marged = answer_df_marged.drop('評価',axis=1)
#出来上がったdfを、類以度でsort　

answer_df_marged_g = answer_df_marged_g.set_index(['総合スコア','企業名'])

#answer_df_marged_g.sort_index(ascending=False)[:top]
answer_df_marged_g.sort_values(by='類似度',ascending=False)[:top]

,,類似度,評価,本文,職種,経験年数,現職/退職,新卒/中途,性別
総合スコア,企業名,,,,,,,,
3.25,株式会社みずほ銀行,0.716424,3.3,ノルマはないと銘打っておるが、かわりに目標がある。達成しないと罰があるわけではないが、人事評価と支店からの空気に耐えられないこともある。とはいえ、かなり高い目標であるので支店の中でも達成できている人は少ないと思う。特に期末月である9月3月は全員死んだ目をしているように思える。社内の風通しは悪くなく、支店長、副支店長も同じフロアで働いているので相談をよくできるし、お客様との面談に同席してもらうこともよくある。支店長は支店のトップであるが目標が未達だと本部から連絡が入り管理されることも。支店の成績が悪いと、賞与の支店への振り分けが少なくなるのでそこがモチベーションになる。事務手続きがかなり煩雑であり手続書はあるが、量が多く、理解しにくいため本部に問い合わせることも多々あるが、急ぎでない場合はメールで問い合わせろと怒られ答えてもらえない場合もよくある。大抵はお客様に聞かれて急ぎの場合がほとんどなので、こまることが多い。,営業,在籍5～10年,現職（回答時）,新卒入社,男性
3.16,SMB建材株式会社,0.706714,2.8,合併するまえの丸紅建材（株）に新卒で入社。丸紅グループということもあり、企業年金、財形貯蓄制度、借上げ社宅制度（実家が遠い人の場合）などは良かった（現在の制度は不明）。しかしながら、業界が木材および商社ということもあり仕事の延長のような飲み会も多くワークライフバランスは取りにくい。部署の直属の先輩・上司の性格によるのかもしれないが、仕事を切り上げ早く帰るような働き方ではなく、休日にも普通にメールが飛び交う状況であった。また、どこの企業でも言えることかもしれないが、社内で全然仕事をしていないような中年男性も多く、若手の仕事に対するモチベーションを削ぐような感じであった。組織としての体制は整っていたと思われる。ただ、稟議書や出張申請、財務分析表など、社内向けの書類作成が多すぎて生産性という観点からは非常に疑問を持たざるおえなかった。,営業,在籍3年未満,退社済み(2020年より前),新卒入社,男性
3.28,第一生命保険株式会社,0.694239,4.3,やはり伝統的な会社なだけあり会社全体の年齢層は高く、古い考え方の人が多い。ただアクチュアリー系の部署は若手が多く、風通しが良いと感じる。会社として今年オフィスカジュアルやテレワークを積極的に導入するなど、これから変わってゆくのだと思う。,アクチュアリー,在籍3年未満,現職（回答時）,新卒入社,男性
4.05,味の素株式会社,0.694059,3.6,長い歴史ｘ食品から、保守的な印象はある。思った通りには早く進まないが、なんだかんだ、前進していることが多い。家族的な会社。中途で入ってきた方もなじむことができれば、生え抜き以上に頭角を現している人も多い。働き方改革も進み、育児中の方や、介護中の方も働きやすい環境が整備されてきた。コアタイム無しのフレックスや、在宅勤務なども進み、働きやすい会社であろう。,間接部門,在籍20年以上,現職（回答時）,新卒入社,男性
3.88,阪和興業株式会社,0.693642,3.4,-風通しのよさ-実力がある人が評価される,営業,在籍3～5年,現職（回答時）,新卒入社,男性
3.18,株式会社カカクコム,0.691427,3.4,非常に働きやすい会社です。公の組織に近いと感じます。個人的に大きく成長できるわけでもないがストレスが非常に少なく、落ち着いて働ける。細く長く働く会社という印象。ルールや社風など大変保守的でスピード感もないため、ベンチャー企業で働いた経験があるような人には物足りないと感じる雰囲気です。オフィスは常にシーンという音が聞こえそうなくらい静かです。中途入社の社員が圧倒的に多い会社なので、中途入社でも働きやすい環境です。ただし、新卒入社の社員がみな大変優秀なので、圧倒されます。,営業,在籍3年未満,現職（回答時）,中途入社,女性
3.48,株式会社インテージ,0.689485,2.5,気に入られた人材は保護してくれて働きやすいと思いますが、気に入られなければ仕事の割り振りを含め評価も悪くなります。残業も多くハードワークなため体力的にきつくなれば女性は辞めていくことも多いです。,営業,在籍5～10年,退社済み(2020年より前),中途入社,女性
2.97,日立造船株式会社,0.688113,3.0,基本的には年功序列型の企業。ある程度の年次までは評価もほぼ一律で、ボーナス等も同期入社の人間ならば横並びになる。,設計,在籍3～5年,現職（回答時）,新卒入社,男性
2.64,THK株式会社,0.687080,2.1,オーナー企業の為、社長第一です。今は社長派と専務派で分かれています。,企画,在籍10～15年,現職（回答時）,新卒入社,男性


In [114]:
results = model.most_similar(positive="若手", topn=10)
for result in results:
    print(result)

('新人', 0.7501629590988159)
('ベテラン', 0.7215745449066162)
('新進', 0.6505381464958191)
('新進気鋭', 0.6488136053085327)
('個性派', 0.6430515646934509)
('中堅', 0.6131073236465454)
('お笑い芸人', 0.6091640591621399)
('気鋭', 0.6067980527877808)
('若手芸人', 0.5996353626251221)
('生え抜き', 0.5959231853485107)
